In [4]:
import numpy as np
import time
import math
import sys
from typing import List, Tuple

def read_tsp_file(filename: str) -> List[Tuple[float, float]]:
    """TSP dosyasını okur ve koordinatları döndürür."""
    coordinates = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        num_cities = int(lines[0].strip())
        for i in range(1, num_cities + 1):
            if i < len(lines):
                values = lines[i].strip().split()
                if len(values) >= 2:
                    x = float(values[0])
                    y = float(values[1])
                    coordinates.append((x, y))
    return coordinates

def calculate_distance(point1: Tuple[float, float], point2: Tuple[float, float]) -> float:
    """İki nokta arasındaki Öklid mesafesini hesaplar."""
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def nearest_neighbor_tsp(coordinates: List[Tuple[float, float]]) -> Tuple[List[int], float]:
    """Nearest Neighbor algoritması ile TSP çözümü."""
    n = len(coordinates)
    visited = [False] * n
    path = [0]  # 0. noktadan başla
    visited[0] = True
    total_distance = 0.0

    # Büyük veri setleri için ilerleme göstergesi
    print(f"Nearest Neighbor algoritması başlatılıyor ({n} şehir)...")
    progress_step = max(1, n // 20)  # %5'lik adımlarla ilerlemeyi göster
    
    for i in range(n - 1):
        if i % progress_step == 0:
            print(f"İlerleme: {i}/{n-1} ({100*i/(n-1):.1f}%)")
            
        last_point = coordinates[path[-1]]
        min_distance = float('inf')
        nearest_index = -1

        for j in range(n):
            if not visited[j]:
                dist = calculate_distance(last_point, coordinates[j])
                if dist < min_distance:
                    min_distance = dist
                    nearest_index = j

        if nearest_index != -1:
            visited[nearest_index] = True
            path.append(nearest_index)
            total_distance += min_distance

    # Son noktadan başlangıç noktasına geri dön
    total_distance += calculate_distance(coordinates[path[-1]], coordinates[path[0]])
    # Başlangıç düğümünü tekrar ekliyoruz (çevrim oluşturmak için)
    path.append(path[0])
    
    print("Nearest Neighbor algoritması tamamlandı!")
    return path, total_distance

def two_opt_improvement(coordinates: List[Tuple[float, float]], initial_path: List[int]) -> Tuple[List[int], float]:
    """2-opt algoritması ile mevcut çözümü iyileştirir."""
    # Başlangıç düğümüne dönüşü (son elemanı) geçici olarak kaldır (işlem kolaylığı için)
    if initial_path[0] == initial_path[-1]:
        path = initial_path[:-1].copy()
    else:
        path = initial_path.copy()
    
    n = len(path)
    improved = True
    iterations = 0
    max_iterations = 100  # Büyük veri seti için daha az iterasyon
    best_distance = float('inf')
    
    # Mevcut toplam mesafeyi hesapla
    current_total_distance = 0.0
    for i in range(n):
        current_total_distance += calculate_distance(coordinates[path[i]], coordinates[path[(i + 1) % n]])
    
    best_distance = current_total_distance
    best_path = path.copy()
    
    print(f"2-opt iyileştirmesi başlatılıyor, başlangıç mesafesi: {current_total_distance:.2f}")
    
    # Büyük veri setleri için daha akıllı 2-opt
    # Tüm olası kenar çiftlerini denemek yerine rastgele örnekleme yapacağız
    while improved and iterations < max_iterations:
        improved = False
        iterations += 1
        
        # Her iterasyonda ne kadar iyileşme sağlandığını izle
        improvement_in_iteration = 0.0
        
        # Büyük veri setleri için daha az kenar çifti dene
        num_edges_to_try = min(10000, n * 20)  # n^2 yerine doğrusal sayıda kenar dene
        
        print(f"İterasyon {iterations}/{max_iterations}, mevcut mesafe: {current_total_distance:.2f}")
        
        # Rastgele kenar çiftlerini seç ve dene
        for _ in range(num_edges_to_try):
            # Rastgele iki indeks seç (i < j olacak şekilde)
            i = np.random.randint(0, n - 2)
            j = np.random.randint(i + 2, n)
            
            # Eğer son ve ilk düğüm arasındaki kenarı değiştirmek istemiyorsak
            if j == n - 1 and i == 0:
                continue
            
            # Mevcut kenarlar: (i,i+1) ve (j,j+1 mod n)
            current_distance = (
                calculate_distance(coordinates[path[i]], coordinates[path[i+1]]) + 
                calculate_distance(coordinates[path[j]], coordinates[path[(j+1) % n]])
            )
            
            # Yeni kenarlar: (i,j) ve (i+1,j+1 mod n)
            new_distance = (
                calculate_distance(coordinates[path[i]], coordinates[path[j]]) + 
                calculate_distance(coordinates[path[i+1]], coordinates[path[(j+1) % n]])
            )
            
            # Eğer yeni kenarlar daha kısaysa, 2-opt değişimi yap
            if new_distance < current_distance:
                # path'in i+1 ile j arasındaki kısmını ters çevir
                path[i+1:j+1] = reversed(path[i+1:j+1])
                delta = current_distance - new_distance
                current_total_distance -= delta
                improvement_in_iteration += delta
                improved = True
        
        print(f"İterasyon {iterations} tamamlandı, iyileşme: {improvement_in_iteration:.2f}")
        
        # En iyi çözümü güncelle
        if current_total_distance < best_distance:
            best_distance = current_total_distance
            best_path = path.copy()
            print(f"Yeni en iyi mesafe: {best_distance:.2f}")
            
        # Eğer bu iterasyonda anlamlı bir iyileşme olmadıysa durabilir
        if improvement_in_iteration < 0.001 * current_total_distance:
            print(f"Yeterli iyileşme yok, iterasyon durduruluyor")
            break
    
    # Başlangıç düğümünü sona ekleyerek çevrimi tamamlayalım
    best_path.append(best_path[0])
    
    # Son mesafeyi doğru hesapla (çevrim olarak)
    final_distance = 0.0
    for i in range(len(best_path) - 1):
        final_distance += calculate_distance(coordinates[best_path[i]], coordinates[best_path[i+1]])
    
    print(f"2-opt tamamlandı, toplam {iterations} iterasyon, son mesafe: {final_distance:.2f}")
    return best_path, final_distance

def print_full_path(path: List[int], distance: float):
    """Tüm tur yolunu ekrana yazdırır."""
    print("\n" + "="*60)
    print("TSP ÇÖZÜM SONUÇLARI")
    print("="*60)
    print(f"Optimal maliyet değeri: {distance:.2f}")
    print("Optimal maliyeti sağlayan path:")
    # Tüm yolu doğrudan yazdır
    path_str = ' -> '.join(map(str, path))
    print(path_str)
    print("="*60)

def main():
    """Ana program akışı - sadece tsp_3038.txt dosyası için çalışır."""
    filename = "tsp_3038.txt"
    
    print(f"TSP Çözücü - {filename} dosyası için")
    print("="*60)
    
    start_time = time.time()
    
    # Dosyayı oku
    print(f"Dosya okunuyor: {filename}")
    try:
        coordinates = read_tsp_file(filename)
        num_cities = len(coordinates)
        print(f"Toplam {num_cities} şehir okundu.")
    except Exception as e:
        print(f"Dosya okuma hatası: {e}")
        return
    
    # Nearest Neighbor ile başlangıç çözümü
    print("\nNearest Neighbor algoritması başlatılıyor...")
    nn_start_time = time.time()
    nn_path, nn_distance = nearest_neighbor_tsp(coordinates)
    nn_time = time.time() - nn_start_time
    
    print(f"Nearest Neighbor çözümü bulundu:")
    print(f"Tur uzunluğu: {nn_distance:.2f}")
    print(f"Hesaplama süresi: {nn_time:.2f} saniye")
    
    # 2-opt ile iyileştirme
    print("\n2-opt iyileştirmesi başlatılıyor...")
    opt_start_time = time.time()
    opt_path, opt_distance = two_opt_improvement(coordinates, nn_path)
    opt_time = time.time() - opt_start_time
    
    print(f"2-opt iyileştirmesi tamamlandı:")
    print(f"Tur uzunluğu: {opt_distance:.2f} (İyileşme: {100*(nn_distance-opt_distance)/nn_distance:.2f}%)")
    print(f"İyileştirme süresi: {opt_time:.2f} saniye")
    
    # Toplam çalışma süresi
    total_time = time.time() - start_time
    print(f"\nToplam çalışma süresi: {total_time:.2f} saniye")
    
    # Tüm yolu ekrana yazdır (dosyaya kaydetmeden)
    print_full_path(opt_path, opt_distance)
    
    return {
        "path": opt_path,
        "distance": opt_distance,
        "time": total_time
    }

if __name__ == "__main__":
    main()

TSP Çözücü - tsp_3038.txt dosyası için
Dosya okunuyor: tsp_3038.txt
Toplam 3038 şehir okundu.

Nearest Neighbor algoritması başlatılıyor...
Nearest Neighbor algoritması başlatılıyor (3038 şehir)...
İlerleme: 0/3037 (0.0%)
İlerleme: 151/3037 (5.0%)
İlerleme: 302/3037 (9.9%)
İlerleme: 453/3037 (14.9%)
İlerleme: 604/3037 (19.9%)
İlerleme: 755/3037 (24.9%)
İlerleme: 906/3037 (29.8%)
İlerleme: 1057/3037 (34.8%)
İlerleme: 1208/3037 (39.8%)
İlerleme: 1359/3037 (44.7%)
İlerleme: 1510/3037 (49.7%)
İlerleme: 1661/3037 (54.7%)
İlerleme: 1812/3037 (59.7%)
İlerleme: 1963/3037 (64.6%)
İlerleme: 2114/3037 (69.6%)
İlerleme: 2265/3037 (74.6%)
İlerleme: 2416/3037 (79.6%)
İlerleme: 2567/3037 (84.5%)
İlerleme: 2718/3037 (89.5%)
İlerleme: 2869/3037 (94.5%)
İlerleme: 3020/3037 (99.4%)
Nearest Neighbor algoritması tamamlandı!
Nearest Neighbor çözümü bulundu:
Tur uzunluğu: 174731.38
Hesaplama süresi: 1.81 saniye

2-opt iyileştirmesi başlatılıyor...
2-opt iyileştirmesi başlatılıyor, başlangıç mesafesi: 174731.